In [ ]:
%run block_swipdg.ipynb

In [ ]:
import numpy as np

In [ ]:
print('estimating error ...')

lambda_bar = lambda_
lambda_hat = lambda_

local_sizes = [block_space.local_space(ii).size() for ii in range(block_space.num_blocks)]
local_starts = [int(np.sum(local_sizes[:ii])) for ii in range(block_space.num_blocks)]
local_starts.append(block_space.mapper.size)
localized_u_h_np = np.array(u_h_vector, copy=False)
localized_u_h_np = [localized_u_h_np[local_starts[ii]:local_starts[ii+1]] for ii in range(block_space.num_blocks)]
localized_u_h = [Vector(block_space.local_space(ii).size(), 0.) for ii in range(block_space.num_blocks)]
for ii in range(block_space.num_blocks):
    tmp = np.array(localized_u_h[ii], copy=False)
    tmp[:] = localized_u_h_np[ii][:]
local_boundary_info = make_subdomain_boundary_info(grid, {'type': 'xt.grid.boundaryinfo.boundarysegmentindexbased',
                                                          'default': 'dirichlet',
                                                          'neumann': '[{} {}]'.format(inner_boundary_id,
                                                                                      inner_boundary_id+1)})

# the local indicators
from dune.gdt import (
    make_ESV2007_nonconformity_product_dd_subdomain_part_dd_subdomain_oversampled_part
        as make_local_nonconformity_product,
    make_ESV2007_residual_product_dd_subdomain_part_leaf_view as make_local_residual_product,
    make_ESV2007_diffusive_flux_product_dd_subdomain_part_leaf_view as make_local_diffusive_flux_product
)

eta_nc_squared = 0.
eta_r_squared = 0.
eta_df_squared = 0.

for ii in range(grid.num_subdomains):
    neighborhood = grid.neighborhood_of(ii)
    neighborhood_space = block_space.restricted_to_neighborhood(neighborhood)

    u_h_restricted_to_neighborhood_vector = neighborhood_space.project_onto_neighborhood(
        [localized_u_h[kk] for kk in neighborhood], neighborhood)
    u_h_restricted_to_neighborhood = make_discrete_function(neighborhood_space,
                                                            u_h_restricted_to_neighborhood_vector)

    def localize(ss):
        return make_discrete_function(neighborhood_space,
                                      neighborhood_space.project_onto_neighborhood(
                                          [localized_u_h[kk] if kk == ss else Vector(block_space.local_space(ss).size(), 0.)
                                           for kk in neighborhood],
                                          neighborhood))
    localized_u_hs = {ss: localize(ss) for ss in neighborhood}

    for jj in neighborhood:
        for kk in neighborhood:
            eta_nc_squared += make_local_nonconformity_product(grid, ii, ii, local_boundary_info,
                                                               lambda_hat, kappa,
                                                               localized_u_hs[jj],
                                                               localized_u_hs[kk],
                                                               over_integrate=2).apply2()

print('  nonconformity indicator: is {} (should be 1.66e-01)'.format(np.sqrt(eta_nc_squared)))